#### * Loading all the required libraries

In [1]:
###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time
from io import StringIO
import io

from selenium import webdriver


#### * Preparing data using web scraping

In [2]:
####
#Author: Surbhi Nayak
#version 1.0

####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=T&d-49653-p=2&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=&orgId=&num1=&state=none&regType=ALL&num2=')


#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]/tbody")

table = browser.find_element_by_css_selector('table.Bordered')
print(table)

print(table)


<ipython-input-2-aa06b08ecc4e>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("chromedriver.exe")


<selenium.webdriver.remote.webelement.WebElement (session="6211aa9055bbc02f2fafea8e11ba8f04", element="976ca4ea-4740-4478-8bd5-2062ad27b28e")>
<selenium.webdriver.remote.webelement.WebElement (session="6211aa9055bbc02f2fafea8e11ba8f04", element="976ca4ea-4740-4478-8bd5-2062ad27b28e")>


<ipython-input-2-aa06b08ecc4e>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = browser.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]/tbody")
<ipython-input-2-aa06b08ecc4e>:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_css_selector('table.Bordered')


#### * Creating Dataframe using scraped data

In [3]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

C:\Users\anike\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:501: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
2,1 CLUB COLOMBIA CARIBE INTERNATIONAL,15-79-51,NO DATA AVAILABLE,NFP,QUEENS,NY
3,1 in 9 The Long Island Breast Cancer Action Co...,05-26-54,113195645,NFP,BALDWIN,NY
4,Academy of General Dentistry Foundation,43-07-85,237310583,NFP,ALBANY,NY
5,ALS ASSOCIATION UPSTATE NEW YORK CHAPTER,21-23-76,412129176,NFP,ELBRIDGE,NY
6,"American Agora Foundation, Inc.",40-62-78,204000236,NFP,NEW YORK,NY
7,AMERICAN CLASSICAL ORCHESTRA INC,15-89-17,061072470,NFP,NEW YORK,NY
8,"AOPA Foundation, Inc.",40-51-26,208817225,NFP,FREDERICK,MD
9,ARNOLD GOLD FOUNDATION,21-38-43,223052098,NFP,ENGLEWOOD CLIFFS,NJ


In [4]:
# Data frame contains blank row at index 0 

df_c = df[1:]
df_c.head(5)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
2,1 CLUB COLOMBIA CARIBE INTERNATIONAL,15-79-51,NO DATA AVAILABLE,NFP,QUEENS,NY
3,1 in 9 The Long Island Breast Cancer Action Co...,05-26-54,113195645,NFP,BALDWIN,NY
4,Academy of General Dentistry Foundation,43-07-85,237310583,NFP,ALBANY,NY
5,ALS ASSOCIATION UPSTATE NEW YORK CHAPTER,21-23-76,412129176,NFP,ELBRIDGE,NY


#### * Creating bucket on s3

In [5]:
# creating bucket in s3
s3_client = boto3.client('s3')
bucket = 'database-bucket-web' # already created on S3
s3_client.create_bucket(Bucket=bucket)

{'ResponseMetadata': {'RequestId': 'TXSPH63H7KN0WHK8',
  'HostId': '8Ud9egKWa/kKEjMwWPvFHREnj2o7gdjCkOE1+200GdBXFVoQjQUeklx9Z+J8izxNBaolA5J7ai8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8Ud9egKWa/kKEjMwWPvFHREnj2o7gdjCkOE1+200GdBXFVoQjQUeklx9Z+J8izxNBaolA5J7ai8=',
   'x-amz-request-id': 'TXSPH63H7KN0WHK8',
   'date': 'Fri, 18 Mar 2022 02:32:29 GMT',
   'location': '/database-bucket-web',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/database-bucket-web'}

#### * Loading data into s3 bucket as CSV file

In [6]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
csv_buffer = StringIO()
df_c.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'charities_bureau_scrape.csv').put(Body=csv_buffer.getvalue()) 

{'ResponseMetadata': {'RequestId': 'TXSHRPP1WA14344P',
  'HostId': 'odkdh/BWCX175norkiVbuO0qSETM9MRBp+A+/PI1rkv4vU2m95HcuMUvxcosJJ48u3PUEKMNgCc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'odkdh/BWCX175norkiVbuO0qSETM9MRBp+A+/PI1rkv4vU2m95HcuMUvxcosJJ48u3PUEKMNgCc=',
   'x-amz-request-id': 'TXSHRPP1WA14344P',
   'date': 'Fri, 18 Mar 2022 02:32:29 GMT',
   'etag': '"5d97ae5ab6bd6d57f9f82c4282f29d3f"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5d97ae5ab6bd6d57f9f82c4282f29d3f"'}

#### * Validating file upload and reading uploaded CSV file

In [7]:
s3 = boto3.resource('s3')
buckets3 = s3.Bucket(bucket)

for obj in buckets3.objects.all():
    key = obj.key
    body = obj.get()['Body'].read()

    if key == 'charities_bureau_scrape.csv':
        print('charities_bureau_scrape.csv file uploaded successfully')
        print('---------------------------csv data as dataframe---------------------------')
        df_s3 = pd.read_csv(io.StringIO(body.decode('utf-8')))
        print(df_s3)

    else:
        print('File not uploaded')


charities_bureau_scrape.csv file uploaded successfully
---------------------------csv data as dataframe---------------------------
    Unnamed: 0                                  Organization Name  NY Reg #  \
0            1                                04/11 10:17 PM test  47-13-95   
1            2               1 CLUB COLOMBIA CARIBE INTERNATIONAL  15-79-51   
2            3  1 in 9 The Long Island Breast Cancer Action Co...  05-26-54   
3            4            Academy of General Dentistry Foundation  43-07-85   
4            5           ALS ASSOCIATION UPSTATE NEW YORK CHAPTER  21-23-76   
5            6                    American Agora Foundation, Inc.  40-62-78   
6            7                   AMERICAN CLASSICAL ORCHESTRA INC  15-89-17   
7            8                              AOPA Foundation, Inc.  40-51-26   
8            9                             ARNOLD GOLD FOUNDATION  21-38-43   
9           10                Balanda Community Association, Inc.  42-90-86   


### References: 

1. https://www.programiz.com/python-programming/working-csv-files
2. https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
3. https://realpython.com/python-boto3-aws-s3/ 
4. https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 